# Protocol of experiments 01.06.2017

__Goals:__
  1. Try out the proposed pipeline (see google slides)
  1. Send file back to today from future.

__Requirements:__
1. Installed [Docker](https://store.docker.com/search?type=edition&offering=community) with the internet access
1. Python3, pip
1. Installed dependencies from requirements.txt 
1. Archive of the [Integron_Finder](https://github.com/gem-pasteur/Integron_Finder/archive/master.zip)

Again, we import the Container class from docker_wrapper module.

In [ ]:
import os

In [ ]:
exec(open('../tooldog/analyse/container.py').read())

As in previous experiment we will define some useful constants and util functions for later use.

In [ ]:
def write_to_file(filename, string = ''):
    """Write string to file

    :param str filename: Filename
    :param str string: String to write
    """
    f = open(filename, 'w')
    f.write(string)
    f.close()

In [ ]:
PYTHON_VERSION = 2 # version of python tool

In [ ]:
TOOL_NAME = 'gem-pasteur-Integron_Finder-4d54052' # name of the tool, assuming it is available via PyPi

In [ ]:
OUTPUT_FORMAT = 'cwl' # output: cwl / xml

In [ ]:
OUTPUT_FILENAME = TOOL_NAME + "." + OUTPUT_FORMAT # output filename

In [ ]:
PYTHON_PATH = "/usr/local/lib/python3.5/dist-packages/" if PYTHON_VERSION == 3 else \
"/usr/local/lib/python2.7/dist-packages/" # PYTHONPATH, required for argparse2tool

In [ ]:
CURRENT_PATH = os.path.realpath(os.getcwd())

Now we are ready to create a container.

In [ ]:
c = Container("inkuzm/tooldog-analyser-sandbox",
              "tail -f /dev/null",  # run until we will stop the container
              environment={'PYTHONPATH': PYTHON_PATH})

Put tarball inside the container. For this we will:
 * Download tarball https://api.github.com/repos/gem-pasteur/Integron_Finder/tarball

In [ ]:
c.put(CURRENT_PATH + "/tool.tar", "/")

Let's create an executional context and a helper to execute the commands in it.

In [ ]:
WORKDIR = "/" + TOOL_NAME

In [ ]:
def cd(path, cmd):
    return "bash -c 'cd " + path + " && " + cmd + "'"

Let's do it!

In [ ]:
cwl_tool = '' 
with c:
    exe1 = c.exec(cd(WORKDIR, "pip2 install ."))
    for line in exe1:
        print(line)
        
    exe2 = c.exec(cd(WORKDIR, "pip2 install argparse2tool"))
    for line in exe2:
        print(line)
        
    exe3 = c.exec(cd(WORKDIR, 'integron_finder --generate_cwl_tool')) # TODO: find a way to get the toolname
    for line in exe3:
        cwl_tool += line.decode("utf-8")[:-1]
        
    write_to_file(OUTPUT_FILENAME, cwl_tool)